Done with Mojo 0.6.0


## Naive Python implementation

In [1]:
%%python
import PIL.Image as Image
import numpy as np

def prepare_toy_image(image_path: str):
    original_image = Image.open(image_path)
    original_width, original_height = original_image.size

    multispectral_image = original_image.resize((original_width//4, original_width//4))
    upscaled_multispectral_image = multispectral_image.resize((original_width, original_height))
    panchromatic_image = original_image.convert("L")

    original_array = np.array(original_image)
    upscaled_multispectral_array = np.array(upscaled_multispectral_image)
    panchromatic_array = np.array(panchromatic_image)

    return original_array, panchromatic_array, upscaled_multispectral_array




For completeness we will define a naive python implementation of the pansharpening procedure.  

In [2]:
%%python 
def naive_pansharpen_python(panchromatic_array: np.ndarray, upscaled_multispectral_array: np.ndarray, weights: np.ndarray, pansharpened_array: np.ndarray):



    for i in range(panchromatic_array.shape[0]):
        for j in range(panchromatic_array.shape[1]):
            psuedo_pan = 0
            sum_weights = 0
            for k in range(upscaled_multispectral_array.shape[-1]):
                psuedo_pan += upscaled_multispectral_array[i, j, k]*weights[k]
                sum_weights += weights[k]

            psuedo_pan = psuedo_pan/sum_weights

            ratio = panchromatic_array[i, j]/psuedo_pan

            for k in range(upscaled_multispectral_array.shape[-1]):
                pansharpened_array[i, j, k] = upscaled_multispectral_array[i, j, k] * ratio
            
    return pansharpened_array

In [3]:
%%python 
from timeit import timeit
from typing import Callable

def benchmark_pansharpen_python(image_path: str, pansharpening_function: Callable, iterations: int=10):

    toy_original_array, toy_panchromatic_array, toy_upscaled_multispectral_array = prepare_toy_image(image_path)
    toy_pansharpened_array = np.zeros_like(upscaled_multispectral_array)
    channel_dependant_luminance_perception = np.array([0.299, 0.587, 0.114])

    secs = timeit(lambda: pansharpening_function(panchromatic_array = toy_panchromatic_array, 
                                                  upscaled_multispectral_array = toy_upscaled_multispectral_array, 
                                                  weights = channel_dependant_luminance_perception, 
                                                  pansharpened_array = toy_pansharpened_array), number=iterations)/iterations
    
    print(secs, "seconds")
    return secs


In [4]:
naive_python_time = benchmark_pansharpen_python("/home/ferdi/Workspace/pansharpen/treed_brain_512.jpeg", naive_pansharpen_python, 2).to_float64()


Error: name 'upscaled_multispectral_array' is not defined


This is, of course, a terrible way to do this. 
Tight loops are extremely slow in Python, and seeing them makes my stomach churn. 

A better way to do this would be to use Numpy, which is written in C++, and which will vectorize some operations

In [5]:
%%python
def pansharpen_numpy(panchromatic_array: np.ndarray, upscaled_multispectral_array: np.ndarray, weights: np.ndarray, pansharpened_array: np.ndarray):
    psuedo_pan_array = np.true_divide((upscaled_multispectral_array*weights).sum(axis=2), weights.sum())
    ratio = panchromatic_array/psuedo_pan_array
    pansharpened_array[:,:,:] = upscaled_multispectral_array * ratio[:,:,None]
    return pansharpened_array

In [6]:
numpy_python_time = benchmark_pansharpen_python("/home/ferdi/Workspace/pansharpen/treed_brain_512.jpeg", pansharpen_numpy, 10).to_float64()


Error: name 'upscaled_multispectral_array' is not defined


In [7]:
print("Numpy is ", naive_python_time/numpy_python_time, " times faster tha naive Python")

error: Execution was interrupted, reason: signal SIGSEGV: address not mapped to object (fault address: 0x0).
The process has been left at the point where it was interrupted, use "thread return -x" to return to the state before expression evaluation.


This is already a huge win, but we had to implicitly rely on highly optimized C++ libraries to get this done.  

Let's first write this in Mojo the same way we did Python. 
As Mojo aims to be a superset of Python, Python code should ideally just work in Mojo. 
At the time of writing for Mojo 0.6.0, this is not 100% true yet, but the aim is that this will be the case when Mojo 1.0.0 is released.  

In [8]:
from python import Python


In [9]:
def prepare_toy_image(image_path: String) -> PythonObject:
    let np = Python.import_module("numpy")
    let Image = Python.import_module("PIL.Image")
    let python_builtins = Python.import_module("builtins")
    original_image = Image.open(image_path)
    original_image_shape = original_image.size
    # Mojo can't take Python ints firectly to Mojo Ints
    original_width  = original_image_shape[0].to_float64().to_int()
    original_height = original_image_shape[1].to_float64().to_int()

    multispectral_image = original_image.resize((original_width//4, original_width//4))
    upscaled_multispectral_image = multispectral_image.resize((original_width, original_height))
    panchromatic_image = original_image.convert("L")

    original_array = np.array(original_image)
    upscaled_multispectral_array = np.array(upscaled_multispectral_image)
    panchromatic_array = np.array(panchromatic_image)

    return python_builtins.tuple(original_array, panchromatic_array, upscaled_multispectral_array)

In [10]:
a = object(10.)
b = object(4.)

#print(a.to_float64() /b.to_float64())

In [11]:
print(a.__pow__(-1)*b)

0.40000000000000002


In [12]:
d = np.zeros((10, 2))

In [13]:
print(d[0][1])

d.__setitem__((1, 1), 5.0)

print(d)

0.0
[[0. 0.]
 [0. 5.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [59]:
def naive_pansharpen_mojo(panchromatic_array: PythonObject, upscaled_multispectral_array: PythonObject, weights: PythonObject, inout pansharpened_array: PythonObject) -> PythonObject:


    for i in range(panchromatic_array.shape[0]):
        print("i", i)
        for j in range(panchromatic_array.shape[1]):
            print("j", j)
            psuedo_pan = object(0)
            sum_weights = object(0)
            for k in range(upscaled_multispectral_array.shape[-1]):
                print("k1", k)
                rhs = upscaled_multispectral_array[i][j][k]*weights[k]
                psuedo_pan +=  rhs.to_float64()
                print("k2", k)
                sum_weights += weights[k].to_float64()
                print("k3", k)
            # objects do not yet implement __truediv__, but they do implement __pow__...
            psuedo_pan = psuedo_pan * sum_weights.__pow__(-1)

            ratio = panchromatic_array[i][j].to_float64() * psuedo_pan.__pow__(-1)
            
            for k in range(upscaled_multispectral_array.shape[-1]):
                print("k4", k)
                pansharpened_array.__setitem__((i, j, k), upscaled_multispectral_array[i][j][k] * PythonObject(ratio))
            
    return pansharpened_array

error: Expression [59]:24:111: cannot construct 'PythonObject' from 'object' value in operator argument
                pansharpened_array.__setitem__((i, j, k), upscaled_multispectral_array[i][j][k] * PythonObject(ratio))
                                                                                                  ~~~~~~~~~~~~^~~~~~~

expression failed to parse (no further compiler diagnostics)

In [43]:
from benchmark import Unit


In [44]:
arrays = prepare_toy_image("/home/ferdi/Workspace/pansharpen/treed_brain_512.jpeg")
toy_original_array = arrays[0]
toy_panchromatic_array = arrays[1]
toy_upscaled_multispectral_array = arrays[2]
shape = toy_upscaled_multispectral_array.shape
toy_pansharpened_array = np.zeros(shape)
channel_dependant_luminance_perception = np.array([0.299, 0.587, 0.114])

In [48]:
naive_pansharpen_mojo(panchromatic_array = toy_panchromatic_array, upscaled_multispectral_array = toy_upscaled_multispectral_array, weights = channel_dependant_luminance_perception, pansharpened_array = toy_pansharpened_array)

i 0
j 0
k1 0
k2 0
k3 0
k1 1
Error: 


In [18]:
def benchmark_matmul_untyped(image_path: String, python_time: Float64, numpy_time: Float64):
    arrays = prepare_toy_image(image_path)
    toy_original_array = arrays[0]
    toy_panchromatic_array = arrays[1]
    toy_upscaled_multispectral_array = arrays[2]
    
    let np = Python.import_module("numpy")
    shape = toy_upscaled_multispectral_array.shape
    toy_pansharpened_array = np.zeros(shape)
    channel_dependant_luminance_perception = np.array([0.299, 0.587, 0.114])

    @parameter
    fn test_fn():
        try:
            _ = naive_pansharpen_mojo(panchromatic_array = toy_panchromatic_array, upscaled_multispectral_array = toy_upscaled_multispectral_array, weights = channel_dependant_luminance_perception, pansharpened_array = toy_pansharpened_array)
        except:
            pass
    print(1)
    let secs = benchmark.run[test_fn](max_runtime_secs=10).mean()
    
    let speedup : Float64 = python_time / secs
    print(secs, "seconds, a", speedup, "x speedup over Python")

In [19]:
benchmark_matmul_untyped("/home/ferdi/Workspace/pansharpen/treed_brain_512.jpeg", naive_python_time, numpy_python_time)

error: Execution was interrupted, reason: signal SIGSEGV: address not mapped to object (fault address: 0x0).
The process has been left at the point where it was interrupted, use "thread return -x" to return to the state before expression evaluation.


In [20]:
from tensor import Tensor, TensorSpec, TensorShape
from utils.index import Index


In [21]:

#let gdal = Python.import_module("osgeo")

In [22]:
original_image = Image.open("/home/ferdi/Workspace/pansharpen/treed_brain_512.jpeg")

In [23]:
original_image_shape = original_image.size
# Mojo can't take Python ints firectly to Mojo Ints
original_width  = original_image_shape[0].to_float64().to_int()
original_height = original_image_shape[1].to_float64().to_int()

In [24]:
original_image.resize((4, 4))

In [25]:
multispectral_image = original_image.resize((original_width//4, original_height//4))
grayscale_image = original_image.convert("L")

In [26]:
original_array = np.array(original_image)
multispectral_array = np.array(multispectral_image)
grayscale_array = np.array(grayscale_image)

In [27]:
upscaled_multispectral_image = multispectral_image.resize((original_width, original_height))
upscaled_multispectral_array = np.array(upscaled_multispectral_image)

In [28]:
channel_dependant_luminance_perception = np.array([0.299, 0.587, 0.114])

In [29]:
var image_tensor = Tensor[DType.uint8](original_height, original_width)


In [30]:
for i in range(original_height):
    for j in range(original_width):
        image_tensor[Index(i, j)] = grayscale_array[i][j].to_float64().to_int()
        

In [31]:
print(Index(10, 15))

(10, 15)


In [32]:
print(image_tensor[0, 600])

69


In [33]:
print(image_tensor[Index(0, 1000)])

68


In [34]:
image_tensor

In [ ]:
def pansharpen(panchromatic_array: np.ndarray, upscaled_multispectral_array: np.ndarray, weights: np.ndarray):
    psuedo_pan_array = np.true_divide((upscaled_multispectral_array*weights).sum(axis=2), weights.sum())
    ratio = grayscale_array/psuedo_pan_array
    new_red = upscaled_multispectral_array[:, :, 0] * ratio
    new_green = upscaled_multispectral_array[:, :, 1] * ratio
    new_blue = upscaled_multispectral_array[:, :, 2] * ratio
    pansharpened_array = np.stack([new_red, new_green, new_blue], axis=2)
    return pansharpened_array